In [ ]:
from cmath import isnan
import os
from random import random
import time
from typing import Any
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer

# read the data from a file
data = pd.read_csv(os.getcwd().replace('code/old_test_code','') + "/data/cleaned_prof_data_v4.csv")

# initialize web driver for selenium
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)


def findMaxCitations(web_elements: list):
    indOfMax = 0
    maxVal = 0
    for i in range(len(web_elements)):
        val = int(web_elements[i].text.split(" - ")[2].split(" ")[2])
        if val > maxVal:
            maxVal = val
            indOfMax = i
        else:
            continue
    return web_elements[indOfMax]


# loop through and find all professors with wrong / no google scholar ID
professors = []
professorExists = True

for i in range(13595, data.shape[0]):

    # initialize values
    scholar_id = data.iat[i, 3],
    citations = data.iat[i, 4],
    h_index = data.iat[i, 5]

    if(data.iat[i, 3] == "NOSCHOLARPAGE" or data.iat[i, 3] == "#NAME?" or isnan(data.iat[i, 4]) or isnan(data.iat[i, 5])):
        print("Name: " + data.iat[i, 0] + " | " + "University: " +
              data.iat[i, 1] + " | " + "id: " + data.iat[i, 3])

        # launch URL
        driver.get("https://scholar.google.com")

        # identify search box
        box = driver.find_element(By.NAME, "q")

        # enter search text
        box.send_keys(data.iat[i, 0] + " " + data.iat[i, 1])

        # make search
        box.send_keys(Keys.ENTER)

        # in the case that the author pops up as a recommended option
        try:
            link = driver.find_element(
                By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]/h4/a")
            link.click()
            time.sleep(random() * 3)
            
        except:
            try:
                # in the case there are multiple options in the recommended table
                list_of_elements = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]").find_elements(By.TAG_NAME, "div")
                print("case 2 was executed")
                max_element = findMaxCitations(list_of_elements)
                max_element.find_element(By.TAG_NAME, "a").click()
                time.sleep(random() * 5)
            except:
                # otherwise loop through all the divs and find the correct professor
                elements_list = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]").find_element(By.CLASS_NAME, "gs_a").find_elements(By.TAG_NAME, "a")
                print(elements_list)
                if elements_list: 
                    for element in elements_list:  # looping through the number of professors in the link
                        try:
                            element.find_element(By.TAG_NAME, "b")
                            element.click()
                            professorExists = True
                            break
                        except NoSuchElementException:
                            professorExists = False
                            continue  # in the case that the element is not found
                    # give time for html / css / js to load
                    time.sleep(random() * 5)
                else: 
                    professorExists = False 

        # retrieve citation, h-index, and scholar_id for that prof
        if professorExists:
            scholar_id = driver.current_url.split("=")[1].split("&")[0]
            citation = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[1]/td[2]").text
            h_index = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[2]/td[2]").text
            time.sleep(random() * 5)
        else:
            scholar_id = None
            citation = None
            h_index = None
        print("scholar_id: " + str(scholar_id) + " h-index: " +
              str(h_index) + " citations: " + str(citation))
        professors.append({
            "Name": data.iat[i, 0],
            "affiliation": data.iat[i, 1],
            "homepage": data.iat[i, 2],
            "scholar-id": scholar_id,
            "citations": citation,
            "h-index": h_index
        })
    else:
        continue


print(len(professors))
df = pd.DataFrame(professors)
df.to_csv("cleaned_prof_data_v5_33.csv")

[WDM] - ====== WebDriver manager ======


2022-07-19 15:49:40,450 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-19 15:49:40,534 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


2022-07-19 15:49:40,537 INFO Get LATEST chromedriver version for 103.0.5060 google-chrome


[WDM] - Driver [/Users/kevins/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


2022-07-19 15:49:40,621 INFO Driver [/Users/kevins/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_63753/3416575854.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Name: Michael Hatzopoulos | University: University of Athens | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Aggeliki Arapoyanni | University: University of Athens | id: NOSCHOLARPAGE
[]
scholar_id: None h-index: None citations: None
Name: Alex Delis | University: University of Athens | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="c8343aa4e7155d0644dae455b82432bc", element="0f1bc8e5-03c7-4050-a241-878852761f74")>, <selenium.webdriver.remote.webelement.WebElement (session="c8343aa4e7155d0644dae455b82432bc", element="d22b6dc6-7b0b-4a62-bf61-a2412630a5c5")>]
scholar_id: None h-index: None citations: None
Name: Clark D. Thomborson | University: University of Auckland | id: jFEDehcAAAAJ
[<selenium.webdriver.remote.webelement.WebElement (session="c8343aa4e7155d0644dae455b82432bc", element="1ca21da7-19b7-4f73-ab95-14faa6f13804")>]
scholar_id: bSpUhd0AAAAJ h-index: 39 citations: 12400
Name: André Nies | University: University of Auck

In [11]:
df

,Name,affiliation,homepage,scholar-id,citations,h-index
0,Peter Slater,University of Alabama - Huntsville,https://www.uah.edu/science/departments/math/f...,None,None,None
1,Huaming Zhang,University of Alabama - Huntsville,https://www.uah.edu/science/departments/comput...,None,None,None
2,Mikel D. Petty,University of Alabama - Huntsville,https://www.uah.edu/images/research/cmsa/pdf/P...,None,None,None
3,Haeyong Chung,University of Alabama - Huntsville,https://www.uah.edu/science/departments/comput...,None,None,None
4,Heggere S. Ranganath,University of Alabama - Huntsville,https://www.uah.edu/science/departments/comput...,None,None,None
5,Daniel Rochowiak,University of Alabama - Huntsville,https://www.uah.edu/science/departments/comput...,None,None,None
6,Sara J. Graves,University of Alabama - Huntsville,https://www.uah.edu/science/departments/comput...,None,None,None
7,Feng W. Zhu,University of Alabama - Huntsville,https://www.uah.edu/science/departments/comput...,None,None,None
8,Li-Yan Yuan,University of Alberta,https://webdocs.cs.ualberta.ca/~yuan,None,None,None
9,Janelle J. Harms,University of Alberta,https://webdocs.cs.ualberta.ca/~harms,None,None,None


In [13]:
professors

[{'Name': 'Shucheng Yu',
  'affiliation': 'University of Arkansas - Little Rock',
  'homepage': 'https://ualr.edu/faculty-7wazt/yu',
  'scholar-id': 'RtatFnQAAAAJ',
  'citations': '1085',
  'h-index': '19'},
 {'Name': 'Mariofanna Milanova',
  'affiliation': 'University of Arkansas - Little Rock',
  'homepage': 'http://ualr.academia.edu/MariofannaMilanova',
  'scholar-id': '0reJ924AAAAJ',
  'citations': '1109',
  'h-index': '16'},
 {'Name': 'Kenji Yoshigoe',
  'affiliation': 'University of Arkansas - Little Rock',
  'homepage': 'https://ualr.edu/eit/2017/02/10/dr-kenji-yoshigoe-2017',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Peiyi Tang',
  'affiliation': 'University of Arkansas - Little Rock',
  'homepage': 'http://www.ualr.edu/pxtang',
  'scholar-id': None,
  'citations': None,
  'h-index': None},
 {'Name': 'Stathes Hadjiefthymiades',
  'affiliation': 'University of Athens',
  'homepage': 'http://cgi.di.uoa.gr/~shadj',
  'scholar-id': None,
  'citations

In [15]:
h_index

54.0

In [3]:
data

,name,affiliation,homepage,scholar_id,citations,h-index
0,P. Vassalos,AUEB,http://pubs.dbs.uni-leipzig.de/aa/aff/Greece/A...,NOSCHOLARPAGE,NaN,NaN
1,Stavros Toumpis,AUEB,https://195.251.255.130/en/faculty/toumpis-sta...,LyiVPKEAAAAJ,2804.0,21.0
2,Ion Androutsopoulos,AUEB,http://www.aueb.gr/users/ion,4UJm5EQAAAAJ,11918.0,42.0
3,Michalis K. Titsias,AUEB,http://www.aueb.gr/users/mtitsias,B-SbkAwAAAAJ,5255.0,30.0
4,Vasilios A. Siris,AUEB,http://www.aueb.gr/users/vsiris,Vj3O0cEAAAAJ,5967.0,31.0
...,...,...,...,...,...,...
20644,Ali C. Begen,Özyeğin University,https://ali.begen.net,ZIZWsXoAAAAJ,5964.0,34.0
20645,Erhan Öztop,Özyeğin University,https://robotics.ozyegin.edu.tr/members/erhan-...,A8YEQMMAAAAJ,3702.0,28.0
20646,Yasar Safkan,Özyeğin University,https://cs.ozyegin.edu.tr/en/faculty/yasarsafkan,NOSCHOLARPAGE,NaN,NaN
20647,Hasan Sözer,Özyeğin University,https://faculty.ozyegin.edu.tr/hsozer,L3FuUX0AAAAJ,847.0,16.0


In [1]:
data.loc[data['name'] == 'Claudio E. Righetti']

NameError: name 'data' is not defined

In [15]:
print(len(professors))
df = pd.DataFrame(professors)
df.to_csv("cleaned_prof_data_v5_32.csv")

16


In [ ]:
professors.append({'Name': 'Serafeim Karampogias',
  'affiliation': 'University of Athens',
  'homepage': 'None',
  'scholar-id': 'None',
  'citations': 'None',
  'h-index': 'None'})